# Rule Controller example

First we import all the used libraries, remember to always import `sinergym` even if it says is not used, because that is needed to define the environments

In [4]:
from typing import List, Any, Sequence
from sinergym.utils.common import get_season_comfort_range
from sinergym.utils.constants import YEAR
from datetime import datetime
import gymnasium as gym
import numpy as np
import sinergym

Now we can define the environment we want to use, in our case we are using the Eplus demo.
Don't forget to deactivate the `flag_normalization`. It is important due to the fact that *MyRuleBasedController*
works with real values, not `[-1,1]` space.

In [6]:
env = gym.make('Eplus-5zone-hot-continuous-v1', flag_normalization=False)

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-continuous-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res3]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment 5zone-hot-continuous-v1 created successfully.


For the Rule-base controller have a look at the already defined controllers, there is one for each building, since the demo is based on the 5Zone building we are extending that controller and defining the action function we desire, feel free to play with the function to define your own action.

In [7]:
from sinergym.utils.controllers import RBC5Zone

class MyRuleBasedController(RBC5Zone):

    def act(self, observation: List[Any]) -> Sequence[Any]:
        """Select action based on outdoor air drybulb temperature and daytime.

        Args:
            observation (List[Any]): Perceived observation.

        Returns:
            Sequence[Any]: Action chosen.
        """
        obs_dict = dict(zip(self.observation_variables, observation))

        out_temp = obs_dict['outdoor_temperature']

        day = int(obs_dict['day_of_month'])
        month = int(obs_dict['month'])
        hour = int(obs_dict['hour'])
        year = int(obs_dict['year'] if obs_dict.get('year',False) else YEAR)

        summer_start_date = datetime(year, 6, 1)
        summer_final_date = datetime(year, 9, 30)

        current_dt = datetime(year, month, day)

        # Get season comfort range
        if current_dt >= summer_start_date and current_dt <= summer_final_date:
            season_comfort_range = self.setpoints_summer
        else:
            season_comfort_range = self.setpoints_summer
        season_comfort_range = get_season_comfort_range(1991,month, day)
        # Update setpoints
        in_temp = obs_dict['air_temperature']

        current_heat_setpoint = obs_dict[
            'htg_setpoint']
        current_cool_setpoint = obs_dict[
            'clg_setpoint']

        new_heat_setpoint = current_heat_setpoint
        new_cool_setpoint = current_cool_setpoint

        if in_temp < season_comfort_range[0]:
            new_heat_setpoint = current_heat_setpoint + 1
            new_cool_setpoint = current_cool_setpoint + 1
        elif in_temp > season_comfort_range[1]:
            new_cool_setpoint = current_cool_setpoint - 1
            new_heat_setpoint = current_heat_setpoint - 1

        #Clip setpoints to the action space
        if new_heat_setpoint>self.env.action_space.high[0]:
            new_heat_setpoint=self.env.action_space.high[0]
        if new_heat_setpoint<self.env.action_space.low[0]:
            new_heat_setpoint=self.env.action_space.low[0]
        if new_cool_setpoint>self.env.action_space.high[1]:
            new_cool_setpoint=self.env.action_space.high[1]
        if new_cool_setpoint<self.env.action_space.low[1]:
            new_cool_setpoint=self.env.action_space.low[1]

        action = (new_heat_setpoint, new_cool_setpoint)
        if current_dt.weekday() > 5 or hour in range(22, 6):
            #weekend or night
            action = (18.33, 23.33)

        return action

Now that we have our controller ready we can use it:

In [8]:

# create rule-based controller
agent = MyRuleBasedController(env)

for i in range(1):
    obs, info = env.reset()
    rewards = []
    terminated = False
    current_month = 0
while not terminated:
    action = agent.act(obs)
    obs, reward, terminated, truncated, info = env.step(action)
    rewards.append(reward)
    if info['month'] != current_month:  # display results every month
        current_month = info['month']
        print('Reward: ', sum(rewards), info)
print(
    'Episode ',
    i,
    'Mean reward: ',
    np.mean(rewards),
    'Cumulative reward: ',
    sum(rewards))

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res3/Eplus-env-sub_run1]
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[ENVIRONMENT] (INFO) : Saving episode output path... [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res3/Eplus-env-sub_run1/output]


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res3/Eplus-env-sub_run1/USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw', '-d', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res3/Eplus-env-sub_run1/output', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res3/Eplus-env-sub_run1/5ZoneAutoDXVAV.epJSON']
[ENVIRONMENT] (INFO) : Episode 1 started.
[SIMULATOR] (INFO) : Handles initialized.
[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
Reward:  -1.483546380748924 {'time_elapsed(hours)': 0.75, 'year': 2003, 'month': 1, 'day': 1, 'hour': 0, 'is_raining': False, 'action': (15.0, 30.0), 'timestep': 2, 'reward': -1.483546380748924, 'reward_energy': -0.11085250479199188, 'reward_comfort': -2.8562402567058562, 'total_energy': 1108.5250479199187, 'abs_comfort': 2.8562402567058562, 'temperatures': [17.143759743294144]}


/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:230: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Reward:  -4330.208931346244 {'time_elapsed(hours)': 744.375, 'year': 2005, 'month': 2, 'day': 1, 'hour': 0, 'is_raining': False, 'action': (15.0, 22.5), 'timestep': 2976, 'reward': -0.005105280431875691, 'reward_energy': -0.010210560863751382, 'reward_comfort': -0.0, 'total_energy': 102.10560863751381, 'abs_comfort': 0.0, 'temperatures': [20.169816897141686]}
Reward:  -5548.886381603512 {'time_elapsed(hours)': 1416.25, 'year': 1995, 'month': 3, 'day': 1, 'hour': 0, 'is_raining': False, 'action': (18.0, 25.5), 'timestep': 5664, 'reward': -0.5485595376450028, 'reward_energy': -0.016317878160147212, 'reward_comfort': -1.0808011971298583, 'total_energy': 163.1787816014721, 'abs_comfort': 1.0808011971298583, 'temperatures': [18.91919880287014]}
Reward:  -8702.910659510862 {'time_elapsed(hours)': 2160.25, 'year': 2002, 'month': 4, 'day': 1, 'hour': 0, 'is_raining': False, 'action': (18.33, 23.33), 'timestep': 8640, 'reward': -0.004536449977537947, 'reward_energy': -0.009072899955075894, 'rew

Always remember to close the environment:

In [9]:
env.close()

[ENVIRONMENT] (INFO) : Environment closed.


.. note:: For more information about our defines controllers and how create a new one, please, visit our [Controller Documentation](https://ugr-sail.github.io/sinergym/compilation/html/pages/controllers.html)